In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ReadPostgres") \
    .config("spark.jars", "/Users/nomantahir/Desktop/pyvm/postgresql-42.7.3.jar") \
    .getOrCreate()

# JDBC config
jdbc_url = "jdbc:postgresql://localhost:5433/sparkdb"
connection_properties = {
    "user": "postgres",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}





25/08/09 14:08:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


load raw data into rawtable staging layer 

In [ ]:
df_customer = spark.read.csv("/Users/nomantahir/Desktop/cources/telecomchurn/RAW_data/WA_Fn-UseC_-Telco-Customer-Churn.csv", header=True, inferSchema=True)
    

df_customer.write.jdbc(
    url=jdbc_url,
    table="telco_customer_churn",
    mode="append",
    properties=connection_properties
)




silver layer
Goal of Silver Layer
Create a clean, enriched version of your raw data, ready for modeling or analytics.
✅ Remove bad data
✅ Handle missing/null values
✅ Drop duplicates
and handel scd

In [2]:
df_raw = spark.read.jdbc(
    url=jdbc_url,
    table="telco_customer_churn",  # raw table
    properties=connection_properties
)



change data types for silver layer and load data into silver layer 


In [16]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim, lower, when
from pyspark.sql.types import DoubleType

# read RAW
df_raw = spark.read.jdbc(
    url=jdbc_url,
    table="telco_customer_churn",
    properties=connection_properties
)

# helper: map yes/no/1/0/true/false -> boolean (case/space tolerant)
def yn_to_bool(colname):
    c = lower(trim(col(colname).cast("string")))
    return when(c.isin('yes','y','1','true'), True) \
           .when(c.isin('no','n','0','false'), False) \
           .otherwise(None)

# build SILVER
df_silver = (
    df_raw
    .withColumn("SeniorCitizen", yn_to_bool("SeniorCitizen"))
    .withColumn("Partner", yn_to_bool("Partner"))
    .withColumn("PhoneService", yn_to_bool("PhoneService"))
    .withColumn("Churn", yn_to_bool("Churn"))
    .withColumn("MonthlyCharges", col("MonthlyCharges").cast(DoubleType()))
    .withColumn(
        "TotalCharges",
        when(trim(col("TotalCharges").cast("string")) == "", None)
        .otherwise(col("TotalCharges").cast(DoubleType()))
    )
    .dropDuplicates(["customerID"])  # optional: avoid dup loads
)

# write to SILVER table (append if table already exists)
df_silver.write.jdbc(
    url=jdbc_url,
    table="silver_telco_customer_churn",
    mode="append",   # or "overwrite" if you intend to fully replace
    properties=connection_properties
)


In [17]:
df_silver.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerid|gender|SeniorCitizen|Partner|dependents|tenure|PhoneService|   multiplelines|internetservice|onlinesecurity|onlinebackup|deviceprotection|techsupport|streamingtv|streamingmovies|      contract|paperlessbilling|       paymentmethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|0002-ORFBO|Female|        false|   true|       Yes|     9|        true|              No|            DSL|            No|         Yes|              No|        Yes|    

25/08/09 00:38:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 936330 ms exceeds timeout 120000 ms
25/08/09 00:38:28 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/09 00:38:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$